In [1]:
import pandas as pd # importing the library

In [2]:
# reading all csv into dataframes
test_task_data = pd.read_csv("test_task.csv")
banned_domains_data = pd.read_csv("banned_domains.csv")
banned_words_data = pd.read_csv("banned_words.csv")
db_domains_data = pd.read_csv("db_domains.csv")
funding_data = pd.read_csv("funding_data.csv", dtype={
    'uuid': 'str',
    'funding_round': 'str',
    'money_raised': 'float64',
    'company_name': 'str'
}, parse_dates=['funding_date'], dayfirst=True) # specifying the datatypes to avoid mixed data types warning

In [3]:
test_task_data.head() # Reading first five rows of the dataframe

,Unnamed: 0,uuid,website,Name,description,valuation_usd,categories,location
0,0,5639c94c-408b-450a-9694-df238f6d9da9,https://www.perplexity.ai,Perplexity AI,Perplexity is a search engine platform that us...,5.200000e+08,"[{'entity_def_id': 'category', 'permalink': 'a...","[{'permalink': 'san-francisco-california', 'uu..."
1,1,00e32424-3cc3-4798-8361-78914c212fb0,https://mistral.ai,Mistral AI,Mistral AI is a platform that assembles team t...,1.999462e+09,"[{'entity_def_id': 'category', 'permalink': 'a...",NaN
2,2,c8326e69-b364-44af-a7a7-1f9239d532f1,https://robinai.co.uk,Robin AI,Robin AI is a legal infrastructure business th...,NaN,"[{'entity_def_id': 'category', 'permalink': 'a...","[{'permalink': 'london-england', 'uuid': 'aad1..."
3,3,cf2c678c-b81a-80c3-10d1-9c5e76448e51,https://www.openai.com,OpenAI,OpenAI is an AI research and deployment compan...,2.900000e+10,"[{'entity_def_id': 'category', 'permalink': 'a...","[{'permalink': 'san-francisco-california', 'uu..."
4,4,0f3fd8ce-9fe3-d6bd-d243-ffdf2fcfb012,http://www.startengine.com,StartEngine,StartEngine is an equity crowdfunding platform...,1.200000e+08,"[{'entity_def_id': 'category', 'permalink': 'c...","[{'permalink': 'los-angeles-california', 'uuid..."


In [4]:
test_task_data.info() # Getting infomation on all the columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     10000 non-null  int64  
 1   uuid           10000 non-null  object 
 2   website        9977 non-null   object 
 3   Name           10000 non-null  object 
 4   description    9832 non-null   object 
 5   valuation_usd  1920 non-null   float64
 6   categories     9985 non-null   object 
 7   location       9995 non-null   object 
dtypes: float64(1), int64(1), object(6)
memory usage: 625.1+ KB


##### 1) Remove all rows with duplicate uuids or websites (leave only one from every duplicate group).

In [5]:
# removing rows with duplicate entities under uuid and website columns
test_task_data = test_task_data.drop_duplicates(subset=["uuid", "website"])

##### 2) Remove the rows with no website, description, category or location

In [6]:
# removing rows with missing values under any of the following columns
test_task_data = test_task_data.dropna(subset=['website', 'description', 'categories', 'location'])

##### 3) Remove all rows where valuation is equal to or exceeds 1 billion dollars.

In [7]:
# removing all rows where where valuation is 1 billion or more
test_task_data = test_task_data[~(test_task_data['valuation_usd'] >= 1_000_000_000)] # using bitwise negation to achieve this

##### 4) Remove the rows with entities which contain uuids or websites from db_domains.csv.

In [8]:
# removing all rows with uuids or websites in db_domains_data

uuids_to_remove = db_domains_data['uuid_cb'].dropna().unique() # getting unique uuids
websites_to_remove = db_domains_data['url'].dropna().unique() # getting unique websites

# filtering the dataframe and removing all rows where uuid or website have values in the extracted uuids and websites from bd_domains_data
test_task_data = test_task_data[~(test_task_data['uuid'].isin(uuids_to_remove) | 
                                          test_task_data['website'].isin(websites_to_remove))] 

##### 5) Remove the rows where the top-level domains of the website are in the banned_domains.csv.

In [9]:
# creating a new column (tld) by extrating the top level domains from the website column
test_task_data['tld'] = test_task_data['website'].str.extract(r'(?:https?://)?(?:www\.)?[^/]+\.(\w+)', expand=False) # using regex

# getting all domains from banned_domains_data datadrame and remoovin the "." that preceeds them into a list
banned_domains = banned_domains_data['urls'].str.replace(r'^\.', '', regex=True).tolist()

# removing rows where the TLD is in the banned domains list
test_task_data = test_task_data[~test_task_data['tld'].isin(banned_domains)]

# droping the just created 'tld' column as it was just temporary
test_task_data = test_task_data.drop(columns=['tld'])

##### 6) Remove the rows with entities the descriptions of which contain any words from banned_words.csv.

In [10]:
# getting all banned words from banned_words_data dataframe into a list
banned_words = banned_words_data['words'].tolist()

# creating a regex pattern that matches any of the banned words (case insensitive)
pattern = '|'.join(banned_words)  # joining the words with '|' to create an OR condition
pattern = r'\b(?:' + pattern + r')\b'  # adding word boundaries for exact matches

# removing all rows where description contains any of the banned words
test_task_data = test_task_data[~test_task_data['description'].str.contains(pattern, case=False, na=False)]

##### 7) The column "categories" contains lists of dictionaries with names of categories for every entity. Create a new column "categories_list" with lists of extracted names of categories from these lists of dictionaries (key 'value').

In [11]:
import ast

# converting the string representation of lists to actual lists of dictionaries
test_task_data['categories'] = test_task_data['categories'].apply(ast.literal_eval)

# creating a new column (categories_list) by extracting the values associated with the "value" key
test_task_data['categories_list'] = test_task_data['categories'].apply(
    lambda x: [category['value'] for category in x] if isinstance(x, list) else []
) # using a lambda function to achieve this

##### 8) The column "location" contains dictionaries with lists of locations at different scale, namely city, region and country, plus continent. Extract cities, regions and countries for every entity and put every list in a separate column named accordingly.

In [12]:
# converting location column into actual lists of dictionaries
test_task_data['location'] = test_task_data['location'].apply(ast.literal_eval)

# defining a function to extract the value based on location_type
def extract_location(locations, loc_type):
    return [loc['value'] for loc in locations if loc['location_type'] == loc_type]

# creating separate columns for city, region, and country
test_task_data['city'] = test_task_data['location'].apply(lambda x: extract_location(x, 'city'))
test_task_data['region'] = test_task_data['location'].apply(lambda x: extract_location(x, 'region'))
test_task_data['country'] = test_task_data['location'].apply(lambda x: extract_location(x, 'country'))
# test_task_data['continent'] = test_task_data['location'].apply(lambda x: extract_location(x, 'continent'))

##### 9) Sum up the investments from funding_data.csv per entity (we recommend using uuid) and add a column in the main dataframe with total investment for every entity based on what you calculated. Not all entities will have investments and not all investment data will be used, so don't worry about blank spaces.

In [13]:
# creating a new dataframe(investment_per_uuid) that contains total money raised per uuid
investment_per_uuid = funding_data.groupby('uuid', as_index=False)['money_raised'].sum()

# merging the two dataframes on the 'uuid' column
test_task_data = test_task_data.merge(investment_per_uuid[['uuid', 'money_raised']], on='uuid', how='left')

# renaming the 'money_raised' column to 'total_investments'
test_task_data['total_investments'] = test_task_data['money_raised']

# dropping the 'money_raised' column
test_task_data.drop(columns=['money_raised'], inplace=True)

In [14]:
test_task_data.tail()

,Unnamed: 0,uuid,website,Name,description,valuation_usd,categories,location,categories_list,city,region,country,total_investments
3316,991,dee4d79d-692a-46fd-bfc4-13109c64efa3,http://circlelabs.xyz/,circle labs,circle labs is a platform that enables anyone ...,NaN,"[{'entity_def_id': 'category', 'permalink': 'a...","[{'permalink': 'san-francisco-california', 'uu...","[Apps, Consumer, Gaming, Virtual Reality]",[San Francisco],[California],[United States],4200000.0
3317,994,5b61e4c8-756a-7a0e-ce42-b3a76da009db,https://gohopscotch.com/,Hopscotch,"Hopscotch is an open, mobile experiences platf...",NaN,"[{'entity_def_id': 'category', 'permalink': 'b...","[{'permalink': 'el-segundo-california', 'uuid'...","[B2B, E-Learning, Edutainment, Financial Servi...",[El Segundo],[California],[United States],NaN
3318,996,7bb0e599-7438-434e-9c45-0ab00fcff7e8,https://pabloair.com,PABLO AIR,PABLO AIR is creating various mobility busines...,NaN,"[{'entity_def_id': 'category', 'permalink': 'a...","[{'permalink': 'incheon-inchon-jikhalsi', 'uui...","[Aerospace, Delivery Service, Drone Management...",[Incheon],[Inch'on-jikhalsi],[South Korea],9140484.0
3319,998,cbe17592-d84d-46d2-a2f4-b19f7041626e,https://spencerhealthsolutions.com,Spencer Health Solutions,Spencer Health Solutions is a developer of an ...,NaN,"[{'entity_def_id': 'category', 'permalink': 'h...","[{'permalink': 'morrisville-north-carolina', '...","[Health Care, Medical Device, Pharmaceutical]",[Morrisville],[North Carolina],[United States],NaN
3320,999,8b0ec269-abfc-4b37-b808-fe2d3b83b78f,https://waste4change.com,Waste4Change,Waste4Change is a leading waste management pla...,NaN,"[{'entity_def_id': 'category', 'permalink': 'c...","[{'permalink': 'jakarta-jakarta-raya', 'uuid':...","[CleanTech, GreenTech, Recycling, Waste Manage...",[Jakarta],[Jakarta Raya],[Indonesia],5000000.0


In [15]:
df = pd.read_csv("test_task.csv")